# Tradingview to vectorbt backtest analysis

In [63]:
from numpy import sort
import vectorbt as vbt
import pandas as pd 
import datetime, pytz
from datetime import datetime, timedelta

# Read In Data

In [64]:
start_date = datetime(2018, 1, 1, tzinfo=pytz.utc)  # time period for analysis, must be timezone-aware
# end_date = datetime.now(pytz.utc)
# end_date = datetime(2020, 1, 1, tzinfo=pytz.utc)
# The following is the number of days to look back for the analysis
# time_buffer = timedelta(days=100)  # buffer before to pre-calculate SMA/EMA, best to set to max window
vbt.settings.portfolio['init_cash'] = 100_000.  # 100,000$
vbt.settings.portfolio['fees'] = 0.0025  # 0.25% # These may already be incorporated into trade file 
# vbt.settings.portfolio['commission'] = 0.0025  # 0.25% May want to use this later

# Read in the Trade File from Tradingview Strategy

In [65]:
# Download trade file from tradingview strategy as csv
trade_file_2hr = "./data/eth_2h_low_dd.csv"

df_trades_2hr = pd.read_csv(trade_file_2hr)
# df_trades_2hr = pd.read_csv(trade_file_90m)

# Download Data to match the same raw price series that you were using in tradingview
The following cell we get binance data to match the tradingview data. Using the vectorbt function allows for you to update your data rather than re-downloading it. Use same frequency and ticker/exchange as what you used in your tradingview strategy for consistency

In [ ]:
# If you would like to update the data, you can do it here
# binance_data_30m.update(show_progress=True)
# # binance_data_2hr.update(show_progress=True)
# print("Updated data")
# binance_data_2hr.save("./data/BINANCE_ETHUSDT_2h.pickle")
# binance_data_30m.save("./data/BINANCE_ETHUSDT_30m.pickle")
# print("Saved data")

In [66]:
symbol = "ETHUSDT"
freq = '30m' 
# Note, binance doesn't support 90m data, so we're will need to do 30 min data and convert
# See here for the constants https://python-binance.readthedocs.io/en/latest/constants.html
binance_data_30m = vbt.BinanceData.download(symbol, start=start_date, end="now UTC", interval='30m', show_progress=True) 
binance_data_2hr = vbt.BinanceData.download(symbol, start=start_date, end="now UTC", interval='2h', show_progress=True)

# Now pull in the 2hour data
try: 
    binance_data_2hr.load("./data/BINANCE_ETHUSDT_2h.pickle")
    print("Loaded 2hr data from file")
except: 
    print("loading data from pickle failed")
    binance_data_2hr.get() # get data from binance first time can comment this out if you want to use the data from the csv
    binance_data_2hr.save("./data/BINANCE_ETHUSDT_2h.pickle") # save data to file this is a pickle file 


# Now pull in the 30m data
try: 
    binance_data_30m.load("./data/BINANCE_ETHUSDT_30m.pickle")
    print("Loaded 30m data from file")
except: 
    print("loading data from pickle failed")
    binance_data_30m.get() # get data from binance first time can comment this out if you want to use the data from the csv
    binance_data_30m.save("./data/BINANCE_ETHUSDT_30m.pickle") # save data to file this is a pickle file 

# Now set the dataframes to the underlying assets
df_data_2hr = binance_data_2hr.data["ETHUSDT"] # get the dataframe from the binance data object
df_data_30m = binance_data_30m.data["ETHUSDT"] # get the dataframe from the binance data object



2018-01-01 00:00:00+00:00 - 2022-03-14 19:30:00+00:00: : 148it [01:47,  1.38it/s]
2018-01-01 00:00:00+00:00 - 2022-03-14 18:00:00+00:00: : 37it [00:27,  1.36it/s]


Loaded 2hr data from file
Loaded 30m data from file


# Create a 90 minute dataframe

In [21]:
print(df_data_30m.columns)



Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Close time', 'Quote volume',
       'Number of trades', 'Taker base volume', 'Taker quote volume'],
      dtype='object')


In [22]:
# Create a dictionary to resample the columns from the 30min dataframe
binance_resample_dict={'Open':'first', 'High':'max', 'Low':'min', 'Close':'last', 'Volume':'sum', 'Close time':'last', 'Quote volume':'sum',
       'Number of trades':'sum', 'Taker base volume':'sum', 'Taker quote volume':'sum'}

df_data_90m = df_data_30m.resample('90min').agg(binance_resample_dict) # resample to 90 min
df_data_90m

,Open,High,Low,Close,Volume,Close time,Quote volume,Number of trades,Taker base volume,Taker quote volume
Open time,,,,,,,,,,
2018-01-01 00:00:00+00:00,733.01,734.52,720.03,729.55,3170.31443,2018-01-01 01:29:59.999000+00:00,2.302778e+06,4269,1725.53798,1.253290e+06
2018-01-01 01:30:00+00:00,729.78,731.90,716.80,724.05,3408.01468,2018-01-01 02:59:59.999000+00:00,2.465686e+06,4677,1764.85829,1.277309e+06
2018-01-01 03:00:00+00:00,723.95,740.00,722.70,738.32,3334.66388,2018-01-01 04:29:59.999000+00:00,2.445694e+06,5435,1691.60727,1.240213e+06
2018-01-01 04:30:00+00:00,738.33,750.00,738.33,748.99,3450.77033,2018-01-01 05:59:59.999000+00:00,2.572894e+06,5254,2116.02902,1.577940e+06
2018-01-01 06:00:00+00:00,748.91,763.55,746.21,755.11,4091.23006,2018-01-01 07:29:59.999000+00:00,3.087202e+06,5724,2268.88235,1.712471e+06
...,...,...,...,...,...,...,...,...,...,...
2022-03-14 13:30:00+00:00,2569.74,2592.68,2555.54,2587.18,33792.55080,2022-03-14 14:59:59.999000+00:00,8.708430e+07,51829,17532.65670,4.518843e+07
2022-03-14 15:00:00+00:00,2587.18,2592.00,2563.00,2576.30,20271.42350,2022-03-14 16:29:59.999000+00:00,5.222165e+07,36507,9743.48920,2.510118e+07
2022-03-14 16:30:00+00:00,2576.29,2578.86,2532.22,2537.03,26124.98810,2022-03-14 17:59:59.999000+00:00,6.661413e+07,43722,12286.29240,3.131951e+07


In [23]:
# Check it out to make sure everything is accurate
df_data_30m

,Open,High,Low,Close,Volume,Close time,Quote volume,Number of trades,Taker base volume,Taker quote volume
Open time,,,,,,,,,,
2018-01-01 00:00:00+00:00,733.01,734.52,720.03,725.51,1007.73288,2018-01-01 00:29:59.999000+00:00,7.313364e+05,1757,562.41552,4.079747e+05
2018-01-01 00:30:00+00:00,725.51,728.61,722.50,727.62,1098.16812,2018-01-01 00:59:59.999000+00:00,7.972228e+05,1357,712.81719,5.174704e+05
2018-01-01 01:00:00+00:00,727.01,732.00,723.00,729.55,1064.41343,2018-01-01 01:29:59.999000+00:00,7.742186e+05,1155,450.30527,3.278448e+05
2018-01-01 01:30:00+00:00,729.78,731.90,716.80,717.97,1241.55743,2018-01-01 01:59:59.999000+00:00,9.015343e+05,1720,585.02986,4.253670e+05
2018-01-01 02:00:00+00:00,717.67,725.75,717.59,720.92,1105.27741,2018-01-01 02:29:59.999000+00:00,7.984038e+05,1625,615.02488,4.443744e+05
...,...,...,...,...,...,...,...,...,...,...
2022-03-14 17:30:00+00:00,2542.54,2547.76,2532.22,2537.03,9837.93530,2022-03-14 17:59:59.999000+00:00,2.497779e+07,15384,4988.98940,1.266576e+07
2022-03-14 18:00:00+00:00,2537.02,2541.48,2517.00,2518.67,11277.39940,2022-03-14 18:29:59.999000+00:00,2.851868e+07,16906,4947.60570,1.251330e+07
2022-03-14 18:30:00+00:00,2518.67,2534.68,2515.00,2529.83,10153.13420,2022-03-14 18:59:59.999000+00:00,2.563524e+07,15425,5537.47920,1.398147e+07


# Fix up the tradingview trade dataframe

In [24]:
# The following only needs to be done once after reading the trades in from tradingview and from vectorbt
# If these are pulled from pickled files then you can comment the following out

df_data_2hr.index = df_data_2hr.index.tz_localize(None) # convert the index to tz-naive
df_trades_2hr.index = pd.to_datetime(df_trades_2hr.index)
df_trades_2hr.index = df_trades_2hr.index.tz_localize(None) # convert the index to tz-naive


In [25]:
df_trades_2hr.index = df_trades_2hr.index.tz_localize(None) # convert the index to tz-naive
df_trades_2hr.index
df_data_2hr.index = df_data_2hr.index.tz_localize(None) # convert the index to tz-naive

In [26]:
# Have a look at the trades dataframe
df_trades_2hr
# Note the date/time should be in UTC and the price should be the "Open" price from our dataframe above or very close to it.

,Trade #,Type,Signal,Date/Time,Price,Contracts,Profit NONE,Profit %,Cum. Profit NONE,Cum. Profit %,Run-up NONE,Run-up %,Drawdown NONE,Drawdown %
1970-01-01 00:00:00.000000000,1,Entry Long,ParLE,2018-01-01 22:00,759.01,79.0222,2997.21,5.00,2997.21,3.00,3009.81,5.02,798.27,1.33
1970-01-01 00:00:00.000000001,1,Exit Long,tpL,2018-01-02 00:00,797.25,79.0222,2997.21,5.00,2997.21,3.00,3009.81,5.02,798.27,1.33
1970-01-01 00:00:00.000000002,2,Entry Long,ParLE,2018-02-17 00:00,938.61,65.8360,1120.60,1.81,4117.81,1.09,2613.84,4.23,350.10,0.57
1970-01-01 00:00:00.000000003,2,Exit Long,ParSE,2018-02-18 02:00,956.01,65.8360,1120.60,1.81,4117.81,1.09,2613.84,4.23,350.10,0.57
1970-01-01 00:00:00.000000004,3,Entry Short,ParSE,2018-02-18 02:00,956.01,66.0511,2848.54,4.51,6966.35,2.74,2892.30,4.58,32.44,0.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1970-01-01 00:00:00.000000695,348,Exit Long,tpL,2022-02-15 08:00,3074.62,95.7252,13942.33,4.97,375742.88,3.02,14001.19,5.00,6803.72,2.43
1970-01-01 00:00:00.000000696,349,Entry Short,ParSE,2022-02-21 12:00,2626.77,108.6675,3952.97,1.38,379695.84,0.83,13718.69,4.81,13543.81,4.74
1970-01-01 00:00:00.000000697,349,Exit Short,Close entry(s) order ParSE,2022-02-22 12:00,2589.35,108.6675,3952.97,1.38,379695.84,0.83,13718.69,4.81,13543.81,4.74
1970-01-01 00:00:00.000000698,350,Entry Long,ParLE,2022-02-22 12:00,2589.35,219.8348,28228.63,4.96,407924.47,5.88,28348.17,4.98,5139.27,0.90


In [27]:
df_data_2hr

,Open,High,Low,Close,Volume,Close time,Quote volume,Number of trades,Taker base volume,Taker quote volume
Open time,,,,,,,,,,
2018-01-01 00:00:00,733.01,734.52,716.80,717.97,4411.87186,2018-01-01 01:59:59.999000+00:00,3.204312e+06,5989,2310.56784,1.678657e+06
2018-01-01 02:00:00,717.67,737.99,717.59,734.50,4327.36175,2018-01-01 03:59:59.999000+00:00,3.146352e+06,6604,2275.46114,1.653412e+06
2018-01-01 04:00:00,734.99,750.00,730.01,748.99,4624.52971,2018-01-01 05:59:59.999000+00:00,3.436388e+06,7042,2712.00358,2.016683e+06
2018-01-01 06:00:00,748.91,763.55,746.21,751.99,4867.81763,2018-01-01 07:59:59.999000+00:00,3.672219e+06,6867,2576.52968,1.944414e+06
2018-01-01 08:00:00,751.77,758.10,737.40,753.21,4202.06918,2018-01-01 09:59:59.999000+00:00,3.146660e+06,5593,2051.95665,1.537156e+06
...,...,...,...,...,...,...,...,...,...,...
2022-03-14 10:00:00,2596.91,2598.97,2576.28,2579.81,24622.97660,2022-03-14 11:59:59.999000+00:00,6.370234e+07,47901,12537.71040,3.243516e+07
2022-03-14 12:00:00,2579.81,2584.49,2555.54,2577.82,33767.21440,2022-03-14 13:59:59.999000+00:00,8.685908e+07,60726,16722.20350,4.301663e+07
2022-03-14 14:00:00,2577.82,2592.68,2566.29,2573.00,33631.09150,2022-03-14 15:59:59.999000+00:00,8.679511e+07,53973,16804.99070,4.337633e+07


In [28]:
# Let's try adding the close price and open prices to the trades dataframe. We might need them later.
df_trades_2hr["Close Price"] = df_data_2hr["Close"]
df_trades_2hr["Open Price"] = df_data_2hr["Open"]



In [29]:
df_data_2hr.index

DatetimeIndex(['2018-01-01 00:00:00', '2018-01-01 02:00:00',
               '2018-01-01 04:00:00', '2018-01-01 06:00:00',
               '2018-01-01 08:00:00', '2018-01-01 10:00:00',
               '2018-01-01 12:00:00', '2018-01-01 14:00:00',
               '2018-01-01 16:00:00', '2018-01-01 18:00:00',
               ...
               '2022-03-14 00:00:00', '2022-03-14 02:00:00',
               '2022-03-14 04:00:00', '2022-03-14 06:00:00',
               '2022-03-14 08:00:00', '2022-03-14 10:00:00',
               '2022-03-14 12:00:00', '2022-03-14 14:00:00',
               '2022-03-14 16:00:00', '2022-03-14 18:00:00'],
              dtype='datetime64[ns]', name='Open time', length=18357, freq=None)

In [30]:
# df_combined  = pd.concat([df_data_90m, df_trades_2hr]).sort_index()
df_combined  = pd.concat([df_data_2hr, df_trades_2hr]).sort_index()

In [31]:
print(df_data_2hr.shape)
print(df_trades_2hr.shape)
test = pd.merge(df_data_2hr, df_trades_2hr, how='left', left_index=True, right_index=True)

(18357, 10)
(700, 16)


In [61]:
test.loc["2018-01-05"]

,Open,High,Low,Close,Volume,Close time,Quote volume,Number of trades,Taker base volume,Taker quote volume,...,Profit NONE,Profit %,Cum. Profit NONE,Cum. Profit %,Run-up NONE,Run-up %,Drawdown NONE,Drawdown %,Close Price,Open Price
Open time,,,,,,,,,,,,,,,,,,,,,
2018-01-05 00:00:00,940.00,981.95,930.10,978.10,6179.46514,2018-01-05 01:59:59.999000+00:00,5.934520e+06,9610,3221.56503,3.095031e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-05 02:00:00,979.14,1004.37,960.00,1003.01,9842.15723,2018-01-05 03:59:59.999000+00:00,9.715743e+06,12434,5304.30433,5.240134e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-05 04:00:00,1003.01,1020.00,1000.00,1008.99,8387.42966,2018-01-05 05:59:59.999000+00:00,8.456631e+06,11207,4335.57588,4.374662e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-05 06:00:00,1009.73,1045.00,1007.25,1030.00,8530.71513,2018-01-05 07:59:59.999000+00:00,8.750038e+06,9874,4878.48108,5.005314e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-05 08:00:00,1029.12,1030.00,970.96,991.87,11935.98567,2018-01-05 09:59:59.999000+00:00,1.195263e+07,14354,5384.53695,5.394031e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-05 10:00:00,993.98,1010.00,980.13,995.98,8245.24166,2018-01-05 11:59:59.999000+00:00,8.177373e+06,10514,3517.82141,3.492501e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-05 12:00:00,995.90,999.99,975.04,978.09,7571.35865,2018-01-05 13:59:59.999000+00:00,7.491330e+06,8926,3385.61122,3.354425e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-05 14:00:00,979.98,983.02,974.00,981.50,6948.77223,2018-01-05 15:59:59.999000+00:00,6.810001e+06,8169,3754.06251,3.679722e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-05 16:00:00,981.11,981.99,931.00,944.80,9176.63455,2018-01-05 17:59:59.999000+00:00,8.796204e+06,11315,3636.71571,3.487356e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
# combine data and trades 

df_combined = pd.merge(df_data_2hr, df_trades_2hr, how='left', left_index=True, right_index=True)

df_combined = df_combined[["Close","Open","Open Price","Price","Type","Contracts"]]
df_combined.loc["2018-01-05"]

,Close,Open,Open Price,Price,Type,Contracts
Open time,,,,,,
2018-01-05 00:00:00,978.10,940.00,NaN,NaN,NaN,NaN
2018-01-05 02:00:00,1003.01,979.14,NaN,NaN,NaN,NaN
2018-01-05 04:00:00,1008.99,1003.01,NaN,NaN,NaN,NaN
2018-01-05 06:00:00,1030.00,1009.73,NaN,NaN,NaN,NaN
2018-01-05 08:00:00,991.87,1029.12,NaN,NaN,NaN,NaN
2018-01-05 10:00:00,995.98,993.98,NaN,NaN,NaN,NaN
2018-01-05 12:00:00,978.09,995.90,NaN,NaN,NaN,NaN
2018-01-05 14:00:00,981.50,979.98,NaN,NaN,NaN,NaN
2018-01-05 16:00:00,944.80,981.11,NaN,NaN,NaN,NaN


In [54]:
# This creates the boolean for the signals
df_combined["entries"] = df_combined["Type"] == "Entry Long"
df_combined["exits"] = df_combined["Type"] == "Exit Long"
df_combined["size"] = df_combined["Contracts"]
df_combined["short_entries"] = df_combined["Type"] == "Entry Short"
df_combined["short_exits"] = df_combined["Type"] == "Exit Short"
df_combined

,Close,Open,Open Price,Price,Type,Contracts,entries,exits,size,short_entries,short_exits
Open time,,,,,,,,,,,
2018-01-01 00:00:00,717.97,733.01,NaN,NaN,NaN,NaN,False,False,NaN,False,False
2018-01-01 02:00:00,734.50,717.67,NaN,NaN,NaN,NaN,False,False,NaN,False,False
2018-01-01 04:00:00,748.99,734.99,NaN,NaN,NaN,NaN,False,False,NaN,False,False
2018-01-01 06:00:00,751.99,748.91,NaN,NaN,NaN,NaN,False,False,NaN,False,False
2018-01-01 08:00:00,753.21,751.77,NaN,NaN,NaN,NaN,False,False,NaN,False,False
...,...,...,...,...,...,...,...,...,...,...,...
2022-03-14 10:00:00,2579.81,2596.91,NaN,NaN,NaN,NaN,False,False,NaN,False,False
2022-03-14 12:00:00,2577.82,2579.81,NaN,NaN,NaN,NaN,False,False,NaN,False,False
2022-03-14 14:00:00,2573.00,2577.82,NaN,NaN,NaN,NaN,False,False,NaN,False,False


In [55]:

df_combined["Close"].fillna(df_combined["Open Price"], inplace=True) # fill in the missing close prices with the open price from the trades dataframe

In [57]:
df_combined.loc["2018-01-05"] # Take a look on a day where multiple trades took place

,Close,Open,Open Price,Price,Type,Contracts,entries,exits,size,short_entries,short_exits
Open time,,,,,,,,,,,
2018-01-05 00:00:00,978.10,940.00,NaN,NaN,NaN,NaN,False,False,NaN,False,False
2018-01-05 02:00:00,1003.01,979.14,NaN,NaN,NaN,NaN,False,False,NaN,False,False
2018-01-05 04:00:00,1008.99,1003.01,NaN,NaN,NaN,NaN,False,False,NaN,False,False
2018-01-05 06:00:00,1030.00,1009.73,NaN,NaN,NaN,NaN,False,False,NaN,False,False
2018-01-05 08:00:00,991.87,1029.12,NaN,NaN,NaN,NaN,False,False,NaN,False,False
2018-01-05 10:00:00,995.98,993.98,NaN,NaN,NaN,NaN,False,False,NaN,False,False
2018-01-05 12:00:00,978.09,995.90,NaN,NaN,NaN,NaN,False,False,NaN,False,False
2018-01-05 14:00:00,981.50,979.98,NaN,NaN,NaN,NaN,False,False,NaN,False,False
2018-01-05 16:00:00,944.80,981.11,NaN,NaN,NaN,NaN,False,False,NaN,False,False


In [37]:
# Show all of the rows where either a long or short entry trade took place
mask = df_combined[(df_combined["short_entries"] == True) | (df_combined["entries"] == True)]
# Create a Mask for simplified searching
mask.index
# Test out the ability to set opening values as main value for the vectorbt strategy, 
new_df = df_combined.loc[mask.index]
new_df["Open"] = df_data_90m["Open"]


In [38]:
new_df

,Close,Open,Open Price,Price,Type,Contracts,entries,exits,size,short_entries,short_exits
1970-01-01 00:00:00.000000000,NaN,NaN,NaN,759.01,Entry Long,79.0222,True,False,79.0222,False,False
1970-01-01 00:00:00.000000002,NaN,NaN,NaN,938.61,Entry Long,65.8360,True,False,65.8360,False,False
1970-01-01 00:00:00.000000004,NaN,NaN,NaN,956.01,Entry Short,66.0511,False,False,66.0511,True,False
1970-01-01 00:00:00.000000006,NaN,NaN,NaN,854.11,Entry Short,75.1414,False,False,75.1414,True,False
1970-01-01 00:00:00.000000008,NaN,NaN,NaN,859.28,Entry Long,74.9387,True,False,74.9387,False,False
...,...,...,...,...,...,...,...,...,...,...,...
1970-01-01 00:00:00.000000690,NaN,NaN,NaN,2983.48,Entry Long,96.5734,True,False,96.5734,False,False
1970-01-01 00:00:00.000000692,NaN,NaN,NaN,2877.54,Entry Short,98.2040,False,False,98.2040,True,False
1970-01-01 00:00:00.000000694,NaN,NaN,NaN,2927.77,Entry Long,95.7252,True,False,95.7252,False,False
1970-01-01 00:00:00.000000696,NaN,NaN,NaN,2626.77,Entry Short,108.6675,False,False,108.6675,True,False


In [39]:
# Show the Long and Short signals
print(df_combined["entries"].value_counts())
print(df_combined["short_entries"].value_counts())


False    18880
True       177
Name: entries, dtype: int64
False    18884
True       173
Name: short_entries, dtype: int64


In [40]:
# Create the portfolio object

portfolio = vbt.Portfolio.from_signals(df_combined["Close"], 
                                        entries=df_combined["entries"].to_list(), 
                                        exits=df_combined["exits"].to_list(), 
                                        size=df_combined["size"].to_list(),
                                        short_entries=df_combined["short_entries"].to_list(), 
                                        short_exits=df_combined["short_exits"].to_list(), 
                                        price=df_combined['Close'].to_list(),
                                        )

In [41]:
# Show the positions and trades in a readable format
portfolio.positions.records_readable

,Position Id,Column,Size,Entry Timestamp,Avg Entry Price,Entry Fees,Exit Timestamp,Avg Exit Price,Exit Fees,PnL,Return,Direction,Status


In [42]:
portfolio.positions.records_readable.columns

Index(['Position Id', 'Column', 'Size', 'Entry Timestamp', 'Avg Entry Price',
       'Entry Fees', 'Exit Timestamp', 'Avg Exit Price', 'Exit Fees', 'PnL',
       'Return', 'Direction', 'Status'],
      dtype='object')

In [43]:
portfolio.positions.records_readable["Direction"]

Series([], Name: Direction, dtype: int64)

In [44]:
# Show the Long and Short signals to the portfolio 
print("VectorBT thinks the portfolio has the following long and short signals: ")
print(portfolio.positions.records_readable["Direction"].value_counts()) # Check the vectorbt portfolio trades 

print("The Input Dataframe had the following information: ")
print("The Long signals are:")
print(df_combined["entries"].value_counts())

print("The Short signals are:")
print(df_combined["short_entries"].value_counts())

print("The original trades from the tradingview file are:")
print(df_trades_2hr["Type"].value_counts())

VectorBT thinks the portfolio has the following long and short signals: 
Series([], Name: Direction, dtype: int64)
The Input Dataframe had the following information: 
The Long signals are:
False    18880
True       177
Name: entries, dtype: int64
The Short signals are:
False    18884
True       173
Name: short_entries, dtype: int64
The original trades from the tradingview file are:
Entry Long     177
Exit Long      177
Exit Short     173
Entry Short    173
Name: Type, dtype: int64


# Let's look at the results from VectorBT

In [45]:
# Show the portfolio statistics and compare the returns of the banchmark
bm_rets_90m = df_combined["Close"].vbt.to_returns() # create returns dataframe for the benchmark
portfolio.stats(settings=dict(benchmark_returns=bm_rets_90m, # Sets the benchmark returns for stats comparisons
                              frequency='90min', # Sets the frequency for stats comparisons
                              )
                )

/Users/ericervin/anaconda3/envs/data-science/lib/python3.8/site-packages/vectorbt/generic/stats_builder.py:397: UserWarning: Metric 'sharpe_ratio' requires frequency to be set
  warnings.warn(warning_message)
/Users/ericervin/anaconda3/envs/data-science/lib/python3.8/site-packages/vectorbt/generic/stats_builder.py:397: UserWarning: Metric 'calmar_ratio' requires frequency to be set
  warnings.warn(warning_message)
/Users/ericervin/anaconda3/envs/data-science/lib/python3.8/site-packages/vectorbt/generic/stats_builder.py:397: UserWarning: Metric 'omega_ratio' requires frequency to be set
  warnings.warn(warning_message)
/Users/ericervin/anaconda3/envs/data-science/lib/python3.8/site-packages/vectorbt/generic/stats_builder.py:397: UserWarning: Metric 'sortino_ratio' requires frequency to be set
  warnings.warn(warning_message)


Start                         1970-01-01 00:00:00
End                           2022-03-14 18:00:00
Period                                      19057
Start Value                              100000.0
End Value                                100000.0
Total Return [%]                              0.0
Benchmark Return [%]                   252.087135
Max Gross Exposure [%]                        0.0
Total Fees Paid                               0.0
Max Drawdown [%]                              NaN
Max Drawdown Duration                         NaN
Total Trades                                    0
Total Closed Trades                             0
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                                  NaN
Best Trade [%]                                NaN
Worst Trade [%]                               NaN
Avg Winning Trade [%]                         NaN
Avg Losing Trade [%]                          NaN


In [46]:
print(portfolio.stats(settings=dict(benchmark_returns=bm_rets_90m, # Sets the benchmark returns for stats comparisons
                              frequency='90min', # Sets the frequency for stats comparisons
                              )
                )
      )
portfolio.plot().show()

/Users/ericervin/anaconda3/envs/data-science/lib/python3.8/site-packages/vectorbt/generic/stats_builder.py:397: UserWarning: Metric 'sharpe_ratio' requires frequency to be set
  warnings.warn(warning_message)
/Users/ericervin/anaconda3/envs/data-science/lib/python3.8/site-packages/vectorbt/generic/stats_builder.py:397: UserWarning: Metric 'calmar_ratio' requires frequency to be set
  warnings.warn(warning_message)
/Users/ericervin/anaconda3/envs/data-science/lib/python3.8/site-packages/vectorbt/generic/stats_builder.py:397: UserWarning: Metric 'omega_ratio' requires frequency to be set
  warnings.warn(warning_message)
/Users/ericervin/anaconda3/envs/data-science/lib/python3.8/site-packages/vectorbt/generic/stats_builder.py:397: UserWarning: Metric 'sortino_ratio' requires frequency to be set
  warnings.warn(warning_message)


Start                         1970-01-01 00:00:00
End                           2022-03-14 18:00:00
Period                                      19057
Start Value                              100000.0
End Value                                100000.0
Total Return [%]                              0.0
Benchmark Return [%]                   252.087135
Max Gross Exposure [%]                        0.0
Total Fees Paid                               0.0
Max Drawdown [%]                              NaN
Max Drawdown Duration                         NaN
Total Trades                                    0
Total Closed Trades                             0
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                                  NaN
Best Trade [%]                                NaN
Worst Trade [%]                               NaN
Avg Winning Trade [%]                         NaN
Avg Losing Trade [%]                          NaN


# Now time to introduce Quantstats tearsheets

In [5]:
import quantstats as qs

In [11]:
portfolio.plot().show()

NameError: name 'portfolio' is not defined

In [8]:
qs.reports.full()

NameError: name 'portfolio' is not defined